In [1]:
%%HTML
<style type="text/css">
    table.dataframe td {
        width: 30px;
        border-style: solid;
        border-width: thin;
        text-align: center; 
        vertical-align: middle;
        border: 1px solid lightgray;
        font-size: 14pt;
    }
</style>

In [2]:
# https://codereview.stackexchange.com/questions/92649/word-search-generator
import pandas as pd
import numpy as np

import random
import string

LETTERS = set(string.ascii_lowercase)
EMPTY = '.'
HARD_DIRECTIONS = dict(up=(-1,0), down=(1,0), left=(0,-1), right=(0,1),
                  upleft=(-1,-1), upright=(-1,1), downleft=(1,-1),
                  downright=(1,1))

MEDIUM_DIRECTIONS = dict(up=(-1,0), down=(1,0), left=(0,-1), right=(0,1))

EASY_DIRECTIONS = dict(down=(1,0), right=(0,1))

DIRECTIONS = EASY_DIRECTIONS

class WordSearch:
    """A word search puzzle.

    Arguments to the constructor:

    width       Width of the puzzle (default: 10).
    height      Height of the puzzle (default: 10).
    word_list   List of words to use (default: None).
    word_file   File to load words from (if word_list is None).
    min_len     Minimum length of words (default: 3).
    max_len     Maximum length of words (default: None).
    directions  Iterable of names of allowed directions (default: all eight).
    density     Stop generating when this density is reached (default: .7).

    """
    def __init__(self, width=10, height=10, word_list=None, word_file=None,
                 min_len=3, max_len=None, directions=DIRECTIONS, density=.7):

        # Check arguments and load word list.
        if max_len is None:
            max_len = min(width, height)
        if word_list is None:
            if word_file is None:
                raise ValueError("neither word_list nor word_file specified")
            word_list = []
            with open(word_file) as f:
                for line in f:
                    word = line.strip()
                    if set(word) <= LETTERS and min_len <= len(word) <= max_len:
                        word_list.append(word)
        else:
            # Take a copy so that we can shuffle it without updating
            # the original.
            word_list = word_list[:]
        random.shuffle(word_list)

        # Initially empty grid and list of words.
        self.grid = [[EMPTY] * width for _ in range(height)]
        self.words = []

        # Generate puzzle by adding words from word_list until either
        # the word list is exhausted or the target density is reached.
        filled_cells = 0
        target_cells = width * height * density
        for word in word_list:
            # List of candidate positions as tuples (i, j, d) where
            # (i, j) is the coordinate of the first letter and d is
            # the direction.
            candidates = []
            for d in directions:
                di, dj = directions[d]
                for i in range(max(0, 0 - len(word) * di),
                               min(height, height - len(word) * di)):
                    for j in range(max(0, 0 - len(word) * dj),
                                   min(width, width - len(word) * dj)):
                        for k, letter in enumerate(word):
                            g = self.grid[i + k * di][j + k * dj]
                            if g != letter and g != EMPTY:
                                break
                        else:
                            candidates.append((i, j, d))
            if candidates:
                i, j, d = random.choice(candidates)
                di, dj = directions[d]
                for k, letter in enumerate(word):
                    if self.grid[i + k * di][j + k * dj] == EMPTY:
                        filled_cells += 1
                        self.grid[i + k * di][j + k * dj] = letter
                self.words.append((word, i, j, d))
                if filled_cells >= target_cells:
                    break

def random_letters(size):
    
    RAND_CHARS = np.array(list(string.ascii_lowercase),dtype=(np.str_, 1))

    nchars = size * size
    rand_string = np.random.choice(RAND_CHARS, nchars)
    
    return pd.DataFrame(rand_string.reshape(size, size))

In [3]:
# wordlist = ['race', 'ice', 'cell', 'city', 'fancy', 'village', 'space', 'bicycle', 'age', 'change', 'edge', 'huge', 'circle', 'spicy', 'badge', 'race', 'bridge', 'dodge', 'fudge', 'change']# wordlist = ['race', 'ice', 'cell', 'city', 'fancy', 'village', 'space', 'bicycle', 'age', 'change', 'edge', 'huge', 'circle', 'spicy', 'badge', 'race', 'bridge', 'dodge', 'fudge', 'change']
wordlist = ['door', 'floor','again','wild','children','climb','parents','most','only','both','gem','giant','giraffe','energy','jacket','jar','jog','join','adjust', 'magic']

In [4]:
ws_easy = WordSearch(
    word_list = wordlist,
    height = 15,
    width = 15,
    directions = EASY_DIRECTIONS
)

ws_medium = WordSearch(
    word_list = wordlist,
    height = 15,
    width = 15,
    directions = MEDIUM_DIRECTIONS
)

ws_hard = WordSearch(
    word_list = wordlist,
    height = 20,
    width = 20,
    directions = HARD_DIRECTIONS
)

In [5]:
def make_worksearch(wordlist, ws: WordSearch, title, savefile=None):
    df = pd.DataFrame(ws.grid)
    dfRandom = random_letters(len(df.columns))
    mask = df != '.'
    imask = df == '.'
    finalWordsearch = df[mask].fillna(dfRandom)
    if savefile: 
        finalWordsearch.to_excel(savefile)
    return finalWordsearch

In [6]:
make_worksearch(wordlist, ws_easy, "27th November 2020 (Year 2: Easy) Spelling Wordsearch", "easy.xlsx")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,k,o,z,p,z,p,a,r,e,n,t,s,x,d,z
1,c,h,i,l,d,r,e,n,e,r,t,b,y,o,j
2,b,d,s,j,g,a,g,a,i,n,h,g,p,o,a
3,l,m,o,s,t,x,r,o,n,l,y,j,n,r,c
4,e,n,e,r,g,y,r,o,a,w,h,p,v,j,k
5,a,b,o,j,u,f,z,q,q,i,i,g,z,s,e
6,c,p,i,a,u,m,c,d,i,l,i,c,k,f,t
7,n,i,a,r,p,a,e,h,l,d,m,z,g,l,c
8,e,a,d,a,c,g,j,g,f,i,o,b,e,o,b
9,f,u,j,j,l,i,o,h,g,z,m,o,m,o,g


In [7]:
make_worksearch(wordlist, ws_medium, "27th November 2020 (Year 2: Medium) Spelling Wordsearch", "medium.xlsx")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,n,g,d,e,c,w,v,r,w,u,t,v,j,y,q
1,m,i,v,n,p,i,o,r,m,o,s,t,a,z,y
2,f,r,v,n,s,l,i,o,n,k,s,u,r,b,k
3,v,a,h,e,o,d,k,o,s,i,c,a,b,c,c
4,p,f,s,r,i,c,j,d,l,v,x,j,l,y,p
5,r,f,w,d,j,o,i,n,d,a,g,a,i,n,l
6,l,e,a,l,n,p,a,r,e,n,t,s,p,h,i
7,x,q,w,i,b,o,t,h,b,i,f,n,c,j,r
8,x,h,r,h,t,s,u,j,d,a,o,j,g,e,o
9,g,q,t,c,z,a,l,t,j,b,d,a,f,n,o


In [8]:
make_worksearch(wordlist, ws_hard, "27th November 2020 (Year 2: Easy) Spelling Wordsearch", "difficult.xlsx")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,o,a,n,m,o,m,f,u,k,h,y,k,x,l,a,n,m,z,k,s
1,x,i,h,h,y,l,n,o,k,e,m,r,l,w,k,l,b,w,l,p
2,g,x,h,j,a,g,i,j,s,u,s,s,b,o,u,u,l,g,l,c
3,h,d,m,e,i,a,r,h,i,h,r,z,q,d,x,r,e,p,h,h
4,z,r,l,y,h,o,h,d,h,f,s,t,o,t,r,m,h,n,g,i
5,l,a,p,i,o,c,s,k,x,j,z,q,u,x,b,r,v,u,o,l
6,q,s,d,l,w,p,d,v,g,t,v,d,o,o,r,a,v,b,j,d
7,y,e,f,a,j,z,c,d,o,g,g,k,t,s,r,j,d,t,l,r
8,w,b,n,a,t,e,k,c,a,j,v,k,t,t,n,b,y,k,b,e
9,k,h,c,e,l,s,n,s,s,i,d,s,c,n,g,s,u,i,d,n


In [9]:
ws_hard.words

[('giant', 16, 15, 'left'),
 ('gem', 2, 17, 'downleft'),
 ('jar', 7, 15, 'up'),
 ('giraffe', 10, 3, 'down'),
 ('magic', 15, 11, 'left'),
 ('wild', 6, 4, 'upleft'),
 ('energy', 7, 1, 'downright'),
 ('again', 13, 10, 'upleft'),
 ('door', 6, 11, 'right'),
 ('jog', 6, 18, 'up'),
 ('floor', 7, 2, 'upright'),
 ('most', 13, 7, 'left'),
 ('parents', 13, 13, 'up'),
 ('join', 18, 6, 'right'),
 ('only', 1, 7, 'left'),
 ('children', 2, 19, 'down'),
 ('adjust', 15, 10, 'upleft'),
 ('both', 19, 14, 'right'),
 ('jacket', 8, 9, 'left'),
 ('climb', 16, 5, 'right')]

In [10]:
print("\n".join([w for (w, _, _, _) in ws_hard.words]))

giant
gem
jar
giraffe
magic
wild
energy
again
door
jog
floor
most
parents
join
only
children
adjust
both
jacket
climb


In [17]:
import json
with open("answers.txt", "w") as f:
    f.write("Notes / Answers.\n=======================================\n")
    f.write("Answers shown as word, row, column (starting at 0) and direction.\n\n")
    f.write("Easy puzzle has words vertical and horizontal (left to right, top to bottom).\n\n")
    f.write("Medium puzzle has words vertical and horizontal (forwards and backwards vertical and horizontal).\n\n")
    f.write("Difficult puzzle has words vertical, horizontal and diagonal (forwards and backwards).\n\n")
    
    f.write("Easy\n")
    f.write(json.dumps(ws_easy.words))
    f.write("\n\nMedium\n")
    f.write(json.dumps(ws_medium.words))
    f.write("\n\nDifficult\n")
    f.write(json.dumps(ws_hard.words))